## Pré-processing suite à détection langue et avant/après traduction?
Accès au dossier du projet sur Google Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/', force_remount=True)
#%cd drive/MyDrive/Projet_Rakuten
%cd G:\.shortcut-targets-by-id\1xn44LDjqEgryLM_F1Lo5CuZZnrJGMnfi\Projet_Rakuten\Output\preprocessing-lang

# Importation des librairies

In [ ]:
import pandas as pd
#from nltk.tokenize import PunktSentenceTokenizer
#from nltk.tokenize import word_tokenize
#from wordcloud import WordCloud #, STOPWORDS

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize.regexp import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Télécharger les ressources de nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('stopwords')



Chargement du fichier source \data\output\preprocessing-lang\X_train_pre.csv

In [ ]:
df = pd.read_csv('X_train_pre.csv', index_col=0)
df.head(10)

# Filtrage stop Words et Tokenization du texte

Envi

In [ ]:
# Création d'une liste personnalisée de stopwords français, anglais et allemends
french_stopwords = set(nltk_stopwords.words('french'))
english_stopwords = set(nltk_stopwords.words('english'))
dutch_stopwords = set(nltk_stopwords.words('dutch'))
combined_stopwords = french_stopwords.union(english_stopwords)
combined_stopwords = combined_stopwords.union(dutch_stopwords)

# Fonction pour nettoyer et extraire les mots, en excluant les stopwords
def extract_words(text):
    # Utilisation d'une expression régulière pour ne conserver que les mots
    words = re.findall(r'\w+', text.lower())
    # Filtrage des mots en excluant les stopwords, les chiffres et les mots de moins de 2 lettres, et suppression des doublons
    unique_words = {word for word in words if word not in combined_stopwords and not word.isdigit() and len(word) > 1}
    return list(unique_words)

# Ajout colonne mots_text contenant les mots de la colonne texte
df['mots_text'] = df['texte'].apply(extract_words)

# Affichage du résultat
display(df.head())
display(df['texte'][0])
df['mots_text'][0]

# Lemmatisation

Fonction pour convertir les tags POS de nltk à ceux compatibles avec WordNetLemmatizer (merci chatGPT)

Fonction qui permet de déterminer s'il faut lemmatiser un verbe, un nom,...

Préparation du lemmatizer

In [ ]:
# Initialisation du lemmatizer
lemmatizer = WordNetLemmatizer()

# Fonction pour lemmatiser une liste de mots
def lemmatize_words(words):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]

Application de la fonction de lemmatisation à la colonne 'mots_text'. ATTENTION CODE > 15 min

In [ ]:
df['mots_lemmatises'] = df['mots_text'].apply(lemmatize_words)

Affichage du DataFrame résultant

In [ ]:
print(df.head(5))

La colonne mots_lemmatises est d'abord convertie en une colonne texte_lemmatise où chaque liste de mots est transformée en une chaîne de caractères.

In [ ]:
# Convertir les listes de mots en chaînes de caractères
df['texte_lemmatise'] = df['mots_lemmatises'].apply(lambda x: ' '.join(x))
df.info()

# Suppression des colonnes superflues

In [ ]:
X = df.drop(['texte', 'mots_text', 'texte', 'mots_lemmatises'],axis=1)
X.rename(columns={'texte_lemmatise': 'texte'}, inplace=True)
X.info()

# Export preprocessed dataframe

In [ ]:
# Identifier les lignes où la colonne 'texte' contient NaN
lignes_avec_nan = X['texte'].isna()

# Afficher les lignes avec NaN
print("\nLignes avec NaN dans la colonne 'texte':")
print(X.loc[lignes_avec_nan])

In [ ]:
X.to_csv("X_preprocess_to_trad.csv")